In [264]:
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import os
import clip
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)


In [265]:
PATH = "/Volumes/ToshibaLouk/DeepFashion"
TRAINING_PATH = "/Volumes/ToshibaLouk/DeepFashion/train_labels.csv"
TEST_PATH = "/Volumes/ToshibaLouk/DeepFashion/test_labels.csv"
CLASSES_PATH = "/Volumes/ToshibaLouk/DeepFashion/classes.txt"
attributes_path_test = '/Volumes/ToshibaLouk/DeepFashion/attributes-test.csv'
attributes_path_train = '/Volumes/ToshibaLouk/DeepFashion/attributes-train.csv'

In [405]:
new_result = result[result['image'].isin(full_df['image_name'].to_list())]
new_result = new_result.rename(columns={"image": "image_name"})

In [414]:
attributes_test = pd.read_csv(attributes_path_test)
category_df = pd.read_csv(TEST_PATH)
category_df = category_df[category_df['image_name'].isin(attributes_test['image_name'].to_list())]
full_df = pd.merge(category_df, attributes_test, on='image_name', how='outer')
full_df = full_df[full_df['image_name'].isin(new_result['image_name'].to_list())]

In [415]:
total_df = pd.merge(full_df, new_result, on='image_name', how='outer')

In [417]:
total_df.to_csv('Img_and_ATTR.csv', index=False)



In [269]:
"Function to change pathnames to MACOS compatible"
#for elem in attributes.index:
#    attributes['image_name'].loc[elem] = attributes.loc[elem]['image_name'].replace('\\', '/')

'Function to change pathnames to MACOS compatible'

In [270]:
#load the data
sample = full_df.sample(n=500, random_state=1)
PATH_img = "/Volumes/ToshibaLouk/DeepFashion"
a_dict = {key: value for (key, value) in enumerate(sample['category_name'].unique())}

In [271]:
text_input = clip.tokenize(sample['category_name'].unique()).to(device)
print(f"Tokens shape: {text_input.shape}")

with torch.no_grad():
    text_features = model.encode_text(text_input).float()
    text_features /= text_features.norm(dim=-1, keepdim=True)
print(f"Text features shape: {text_features.shape}")

Tokens shape: torch.Size([32, 77])
Text features shape: torch.Size([32, 512])


In [272]:
sample["prediction"] = np.nan
sample["prediction"] = pd.Series([], dtype=object)

sample["top-3"] = np.nan
sample["top-3"] = pd.Series([], dtype=object)

for elem in sample.index:
    path = PATH_img + '/' + sample['image_name'][elem]
    #img = mpimg.imread(path)
    #imgplot = plt.imshow(img)
    #plt.show()
    img = Image.open(path)
    image_input = preprocess(img).unsqueeze(0).to(device)
    
    with torch.no_grad():
        image_features = model.encode_image(image_input).float()
        
    # Pick the top 5 most similar labels for the image
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    values, indices = similarity[0].topk(1)
    values_3, indices_3 = similarity[0].topk(3)
    
    prediction = indices.numpy()[0]
    top_3 = indices_3.numpy()
    #print(top_3)
    
    sample['prediction'].loc[elem] = int(prediction)
    sample['top-3'].loc[elem] = top_3
    
sample = sample.replace({"prediction": a_dict})

from sklearn.metrics import confusion_matrix, accuracy_score
df_confusion = pd.crosstab(sample['category_name'], sample['prediction'])
acc = accuracy_score(sample['category_name'], sample['prediction'])

In [273]:
#calculating the top 3 accuracy

for elem in sample.index:
    a_list = sample['top-3'].loc[elem]
    new_list = []
    for i in a_list:
        val = a_dict[i]
        new_list.append(val)
    sample['top-3'].loc[elem] = new_list
    
sample["top_3_pred"] = np.nan
for elem in sample.index:
    if sample['category_name'].loc[elem] in sample['top-3'].loc[elem]:
        sample["top_3_pred"].loc[elem] = sample['category_name'].loc[elem]
    
    else:
        sample['top_3_pred'].loc[elem] = sample['top-3'].loc[elem][0]
        
top_3_acc = accuracy_score(sample['category_name'], sample['top_3_pred'])
top_3_acc

/Users/louk/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


0.656

In [274]:
pd.set_option('display.max_columns', None)
df_confusion

prediction,Blazer,Blouse,Button-Down,Caftan,Cardigan,Chinos,Coat,Culottes,Cutoffs,Dress,Flannel,Henley,Hoodie,Jacket,Jeans,Joggers,Jumpsuit,Leggings,Parka,Poncho,Romper,Shorts,Skirt,Sweater,Sweatpants,Sweatshorts,Tee
category_name,,,,,,,,,,,,,,,,,,,,,,,,,,,
Blazer,7,4,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
Blouse,0,43,1,1,1,0,1,2,1,1,1,0,0,0,0,0,1,1,0,1,2,0,0,1,0,0,1
Bomber,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
Button-Down,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Caftan,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Cardigan,1,3,0,1,8,0,4,0,0,0,0,0,0,1,0,0,0,1,0,3,1,0,0,3,0,0,0
Chinos,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Coat,0,0,0,1,0,0,3,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Culottes,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [275]:
#most confused classes
most_confused = []
for elem in df_confusion.index:    
    alist = list(zip(df_confusion.loc[elem].index.to_list(), df_confusion.loc[elem].values.tolist()))
    alist.sort(key=lambda x: x[1], reverse=True)
    top_3 = alist[0:3]
    most_confused.append((elem, top_3))

most_confused = sorted(most_confused, key=lambda tup: tup[1][0][1], reverse=True)

[(elem[0], [(x[0], x[1]) for x in elem[1] if x[0] != elem[0] and x[1] > 3 ]) for elem in most_confused]

[('Dress', [('Caftan', 19), ('Romper', 13)]),
 ('Blouse', []),
 ('Tee', [('Blouse', 14), ('Dress', 5)]),
 ('Shorts', [('Culottes', 17), ('Cutoffs', 5)]),
 ('Tank', [('Blouse', 12), ('Cutoffs', 11)]),
 ('Jacket', [('Coat', 5)]),
 ('Skirt', [('Culottes', 9)]),
 ('Cardigan', [('Coat', 4)]),
 ('Sweater', [('Blouse', 4)]),
 ('Top', [('Blouse', 8)]),
 ('Blazer', [('Blouse', 4)]),
 ('Cutoffs', []),
 ('Hoodie', []),
 ('Jumpsuit', []),
 ('Romper', [('Dress', 4)]),
 ('Joggers', [('Culottes', 4)]),
 ('Leggings', []),
 ('Coat', []),
 ('Jeans', []),
 ('Kimono', []),
 ('Poncho', []),
 ('Sweatpants', []),
 ('Bomber', []),
 ('Button-Down', []),
 ('Caftan', []),
 ('Chinos', []),
 ('Culottes', []),
 ('Flannel', []),
 ('Henley', []),
 ('Parka', []),
 ('Sweatshorts', []),
 ('Trunks', [])]

In [276]:
text_input_style = clip.tokenize(['formal', 'sporty', 'trendy', 'classic']).to(device)
print(f"Tokens shape: {text_input.shape}")

with torch.no_grad():
    text_features = model.encode_text(text_input_style).float()
    text_features /= text_features.norm(dim=-1, keepdim=True)
print(f"Text features shape: {text_features.shape}")

sample["prediction_style"] = np.nan
sample["prediction_style"] = pd.Series([], dtype=object)

for elem in sample.index:
    path = PATH_img + '/' + sample['image_name'][elem]
    #img = mpimg.imread(path)
    #imgplot = plt.imshow(img)
    #plt.show()
    img = Image.open(path)
    image_input = preprocess(img).unsqueeze(0).to(device)
    
    with torch.no_grad():
        image_features = model.encode_image(image_input).float()
        
    # Pick the top 5 most similar labels for the image
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    values, indices = similarity[0].topk(1)
    values_3, indices_3 = similarity[0].topk(3)
    
    prediction = indices.numpy()[0]
    top_3 = indices_3.numpy()
    #print(top_3)
    
    sample['prediction_style'].loc[elem] = int(prediction)
    

Tokens shape: torch.Size([32, 77])
Text features shape: torch.Size([4, 512])


In [277]:
style_dict =  {key: value for (key, value) in enumerate(['formal', 'sporty', 'trendy'])}
sample = sample.replace({"prediction_style": style_dict})

In [278]:
sample

,image_name,category_name,labels,prediction,top-3,top_3_pred,prediction_style
19153,img/Flounced_Floral_Dress/img_00000032.jpg,Dress,"[floral, print]",Caftan,"[Caftan, Poncho, Coat]",Caftan,trendy
13,img/Sheer_Pleated-Front_Blouse/img_00000096.jpg,Blouse,[sheer],Blouse,"[Blouse, Romper, Button-Down]",Blouse,trendy
438,img/Tile_Print_Blouse/img_00000015.jpg,Blouse,[print],Poncho,"[Poncho, Sweater, Caftan]",Poncho,trendy
17186,img/Faux_Leather_Fit_&_Flare_Dress/img_0000001...,Dress,"[faux-leather, leather]",Dress,"[Dress, Romper, Skirt]",Dress,formal
13933,img/Elephant_Print_Drawstring_Shorts/img_00000...,Shorts,[print],Poncho,"[Poncho, Blouse, Caftan]",Poncho,trendy
...,...,...,...,...,...,...,...
4602,img/Abstract_Paisley_Boxy_Top/img_00000043.jpg,Top,[summer],Blouse,"[Blouse, Poncho, Button-Down]",Blouse,trendy
7501,img/Basic_Athletic_Racerback_Tank/img_00000034...,Tank,[basic],Tee,"[Tee, Blouse, Jumpsuit]",Tee,sporty
945,img/Classic_Single-Button_Blazer/img_00000066.jpg,Blazer,[satin],Blazer,"[Blazer, Jacket, Button-Down]",Blazer,trendy
7725,img/Hooded_Stripe_Poncho/img_00000029.jpg,Poncho,"[striped, woven]",Poncho,"[Poncho, Caftan, Blouse]",Poncho,trendy


In [292]:
attribute_classes = ['abstract-print', 'animal', 'baroque', 'basic', 'beach', 'bird-print', 'boho', 'botanical-print', 'camouflage', 'cargo', 'chic', 'chiffon', 
                     'circle', 'civil', 'colorblock', 'cotton', 'cozy', 'crochet', 'cute', 'denim', 'dotted', 'elegant', 'everyday', 'fancy', 'faux-fur', 'faux-leather',
                     'faux-suede', 'feather', 'floral', 'fur', 'glitter', 'graphic', 'grid-print', 'grunge', 'knotted', 'leaf-print', 'leather', 'leopard-print', 'linen',
                     'linen-blend', 'logo', 'luxe', 'marble-print', 'medallion-print', 'mesh', 'metallic', 'mixed-print', 'multi-stripe', 'neon', 'neoprene', 'nets', 
                     'netted', 'nylon', 'oil', 'ombre', 'organza', 'ornate-print', 'paisley-print', 'palm-print', 'party', 'patched', 
                     'pattern', 'pleated', 'print', 'relaxed', 'retro', 'safari', 'sateen', 'satin', 'sheer', 'smart', 'soft', 'solid', 'sophisticated', 'sparkling', 
                     'sporty', 'springs', 'square', 'star','strap', 'stretch', 'striped', 'suede', 'summer', 'sweet', 'textured', 'thermal', 'tie-dye', 'training', 
                     'triangle', 'tropical', 'tweed', 'utility', 'velvet', 'weekend', 'workout', 'woven', 'zigzag']

attribute_print = ['floral', 'graphic', 'striped', 'embroidered', 'pleated', 'solid', 'lattice']
attribute_form = ['long-sleeve', 'short-sleeve', 'sleeveless']
attribute_material = ['denim', 'chiffon', 'cotton', 'leather', 'faux', 'knit']
attribute_fit = ['tight', 'loose', 'conventional']


text_input_attributes = clip.tokenize(attribute_classes).to(device)

with torch.no_grad():
    text_features = model.encode_text(text_input_attributes).float()
    text_features /= text_features.norm(dim=-1, keepdim=True)
print(f"Text features shape: {text_features.shape}")

sample["prediction_attributes"] = np.nan
sample["prediction_attributes"] = pd.Series([], dtype=object)

for elem in sample.index:
    path = PATH_img + '/' + sample['image_name'][elem]
    #img = mpimg.imread(path)
    #imgplot = plt.imshow(img)
    #plt.show()
    img = Image.open(path)
    image_input = preprocess(img).unsqueeze(0).to(device)
    
    with torch.no_grad():
        image_features = model.encode_image(image_input).float()
        
    # Pick the top 5 most similar labels for the image
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    
    
    values, indices = similarity[0].topk(len(sample['labels'].loc[elem]))
    
    top_x = indices.numpy()
    #print(top_3)
    
    sample['prediction_attributes'].loc[elem] = top_x

    


Text features shape: torch.Size([98, 512])


In [295]:
attribute_dict =  {key: value for (key, value) in enumerate(attribute_classes)}
for elem in sample.index:
    sample['prediction_attributes'].loc[elem] = np.vectorize(attribute_dict.get)(sample['prediction_attributes'].loc[elem])

In [298]:
sample[['labels', 'prediction_attributes']].head(50)

,labels,prediction_attributes
19153,"[floral, print]","[sateen, organza]"
13,[sheer],[sateen]
438,[print],[mixed-print]
17186,"[faux-leather, leather]","[sateen, grid-print]"
13933,[print],[boho]
4280,[striped],[sporty]
4230,[solid],[sateen]
10944,"[denim, stretch]","[sateen, colorblock]"
4403,[print],[abstract-print]
8005,"[chiffon, print]","[grid-print, paisley-print]"


In [ ]:
sample['labels'].loc[19153]
#img = mpimg.imread(PATH  + '/' + sample[sample['category_name'] == 'Jeggings']['image_name'])

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
img = mpimg.imread(PATH  + '/' + sample['image_name'].loc[])
imgplot = plt.imshow(img)
plt.xlabel(sample['prediction_style'].loc[13933])
plt.show()

## GPT-3

In [ ]:
!pip install openai


In [ ]:
import json
import openai


0                img/Sweet_Crochet_Blouse/img_00000070.jpg
1                img/Classic_Pencil_Skirt/img_00000010.jpg
2        img/Strapless_Diamond_Print_Dress/img_00000038...
3        img/Mid-Rise_-_Acid_Wash_Skinny_Jeans/img_0000...
4        img/Zippered_Single-Button_Blazer/img_00000078...
                               ...                        
13995    img/Baroque_Print_High-Neck_Romper/img_0000002...
13996    img/Crocheted_Mandala_Print_Dress/img_00000007...
13997       img/Mesh_Contrast-Pocket_Tank/img_00000031.jpg
13998             img/Classic_Toggle_Coat/img_00000025.jpg
13999         img/Lace-Paneled_Maxi_Dress/img_00000049.jpg
Name: 0, Length: 14000, dtype: object

In [377]:
df_1 = pd.read_csv('/Volumes/ToshibaLouk/Attributes/train.txt', delimiter = "\t", header=None)
df_2 = pd.read_csv('/Volumes/ToshibaLouk/Attributes/train_attr.txt', delimiter = "\t", header=None)
df_3 = pd.read_csv('/Volumes/ToshibaLouk/Attributes/test.txt', delimiter = "\t", header=None)
df_4 = pd.read_csv('/Volumes/ToshibaLouk/Attributes/test_attr.txt', delimiter = "\t", header=None)

In [378]:
for elem in df_4.index:
    df_4[0].loc[elem] = df_4[0].loc[elem].split(" ")[:-1]

for elem in df_2.index:
    df_2[0].loc[elem] = df_2[0].loc[elem].split(" ")[:-1]

In [379]:
df_2['image'] = df_1[0]

In [380]:
df_4['image'] = df_3[0]

In [381]:
result = pd.concat([df_2, df_4])

In [382]:
result = result.reset_index(drop=True)

In [383]:
column_names = ['floral', 'graphic', 'striped', 'embroidered', 'pleated', 'solid', 'lattice','long-sleeve', 'short-sleeve', 'sleeveless',
                   'maxi_length', 'mini_length', 'no_dress','crew_neckline', 'v_neckline', 'square_neckline', 'no_neckline','denim', 'chiffon', 'cotton', 'leather', 'faux', 'knit',
                   'tight', 'loose', 'conventional']

In [384]:
#result[0].loc[0][column_names.index('graphic')]

In [385]:
for elem in column_names:
    result[elem] = np.nan

In [386]:
for elem in result.index:
    for name in column_names:
        result[name].loc[elem] = result[0].loc[elem][column_names.index(name)]

/Users/louk/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [388]:
result = result.drop([0], axis=1)

In [389]:
result

,image,floral,graphic,striped,embroidered,pleated,solid,lattice,long-sleeve,short-sleeve,sleeveless,maxi_length,mini_length,no_dress,crew_neckline,v_neckline,square_neckline,no_neckline,denim,chiffon,cotton,leather,faux,knit,tight,loose,conventional
0,img/Sweet_Crochet_Blouse/img_00000070.jpg,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1
1,img/Classic_Pencil_Skirt/img_00000010.jpg,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0
2,img/Strapless_Diamond_Print_Dress/img_00000038...,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1
3,img/Mid-Rise_-_Acid_Wash_Skinny_Jeans/img_0000...,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,1,0,0
4,img/Zippered_Single-Button_Blazer/img_00000078...,0,0,0,0,0,1,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17995,img/Rose_Print_Cardigan/img_00000024.jpg,1,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1
17996,img/Tile_Print_Jumpsuit/img_00000013.jpg,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1
17997,img/Side_Vent_Heathered_Tank/img_00000006.jpg,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0
17998,img/Heathered_Curved-Hem_Tank/img_00000014.jpg,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,1


,image_name,category_name,labels,prediction,top-3,top_3_pred,prediction_style,prediction_attributes
19153,img/Flounced_Floral_Dress/img_00000032.jpg,Dress,"[floral, print]",Caftan,"[Caftan, Poncho, Coat]",Caftan,trendy,"[sateen, organza]"
13,img/Sheer_Pleated-Front_Blouse/img_00000096.jpg,Blouse,[sheer],Blouse,"[Blouse, Romper, Button-Down]",Blouse,trendy,[sateen]
438,img/Tile_Print_Blouse/img_00000015.jpg,Blouse,[print],Poncho,"[Poncho, Sweater, Caftan]",Poncho,trendy,[mixed-print]
17186,img/Faux_Leather_Fit_&_Flare_Dress/img_0000001...,Dress,"[faux-leather, leather]",Dress,"[Dress, Romper, Skirt]",Dress,formal,"[sateen, grid-print]"
13933,img/Elephant_Print_Drawstring_Shorts/img_00000...,Shorts,[print],Poncho,"[Poncho, Blouse, Caftan]",Poncho,trendy,[boho]
...,...,...,...,...,...,...,...,...
4602,img/Abstract_Paisley_Boxy_Top/img_00000043.jpg,Top,[summer],Blouse,"[Blouse, Poncho, Button-Down]",Blouse,trendy,[sateen]
7501,img/Basic_Athletic_Racerback_Tank/img_00000034...,Tank,[basic],Tee,"[Tee, Blouse, Jumpsuit]",Tee,sporty,[sateen]
945,img/Classic_Single-Button_Blazer/img_00000066.jpg,Blazer,[satin],Blazer,"[Blazer, Jacket, Button-Down]",Blazer,trendy,[grid-print]
7725,img/Hooded_Stripe_Poncho/img_00000029.jpg,Poncho,"[striped, woven]",Poncho,"[Poncho, Caftan, Blouse]",Poncho,trendy,"[boho, pleated]"


In [1]:


subsample = sample.sample(n=10, random_state=1)

import numpy as np
import matplotlib.pyplot as plt

w=10
h=10
fig=plt.figure(figsize=(8, 8))
columns = 4
rows = 5
for i in range(1, columns*rows +1):
    img = mpimg.imread(PATH  + '/' + sample['image_name'].loc[15958])
    plt.xlabel(sample['prediction_style'].loc[15958])
    fig.add_subplot(rows, columns, i)
    plt.imshow(img)
    
plt.show()

NameError: name 'sample' is not defined